In [1]:
# Set parameters for testing
# This code cell will be replaced by Times Square
# /sdf/home/h/huanlin/shared_campaigns_LSST/DM-53195/submit/LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195
#submitHead = "/sdf/data/rubin/shared/campaigns/LSSTCam-Nightly-Validation-CC/"
#tasksHead = "DM-50157_20250609/submit/LSSTCam/runs/nightlyValidation/20250609/d_2025_06_09/DM-50157"
#submitHead = "/sdf/home/h/huanlin/shared_campaigns_LSST/"
submitHead = "/home/h/huanlin/shared_campaigns_LSST/"
tasksHead = "DM-53195/submit/LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195/"

In [2]:
# --------------------------------------------------------------------------------------
# failureLister - get list of failed task jobs and show the errors
# author: Homer Neal
# ---------------------------------------------------------------------------------------
import sys
import numpy as np
import time
import subprocess
import re
import math
import glob, os
import pandas as pd
import json
from tabulate import tabulate
from IPython.display import display, HTML

# ---- types
#analyzeSingleVisitStar
#analyzeSingleVisitStarAssociation
#analyzeSingleVisitStarAstrometricRefMatch
#analyzeSingleVisitStarPhotometricRefMatch
#associateIsolatedStar
#coadd
#consolidateHealSparsePropertyMaps
#consolidateSingleVisitStar
#consolidateVisitSummary
#makeAnalysisSingleVisitStarAstrometricRefMatch
#makeAnalysisSingleVisitStarPhotometricRefMatch
#makeHealSparsePropertyMaps
#makeWarpTract
#selectDeepCoaddVisits

jobs = ["generateEphemerides","fgcmFitCycle","analyzeSingleVisitStar","analyzeSingleVisitStarAssociation","analyzeSingleVisitStarAstrometricRefMatch","analyzeSingleVisitStarPhotometricRefMatch","associateIsolatedStar","coadd","consolidateHealSparsePropertyMaps","consolidateSingleVisitStar","consolidateVisitSummary","finalJob","makeAnalysisSingleVisitStarAssociationMetricTable","makeAnalysisSingleVisitStarAssociationWholeSkyPlot","makeAnalysisSingleVisitStarAstrometricRefMatch","makeAnalysisSingleVisitStarPhotometricRefMatch","makeHealSparsePropertyMaps","makeInitialVisitDetectorTable","makeInitialVisitTable","makeWarpTract","pipetaskInit","plotPropertyMapSurvey","selectDeepCoaddVisits","step1detector"]
#jobs = ["analyzeSingleVisitStarAssociation"]
#jobs = ["step1detector"]

max_step1 = 100


def dumpError(flin):
    err = ""
    fpin = open(flin)
    for ln in fpin:
        if "ERROR" in ln.upper() :
            err = ln
            break
    return err

# tasksHead = "LSSTCam/runs/nightlyValidation/20250530/d_2025_05_30/DM-50157/"
# tasksHead = "DM-50157_20250607/submit/LSSTCam/runs/nightlyValidation/20250607/d_2025_06_08/DM-50157/"

leadPath = submitHead+tasksHead
os.chdir(leadPath)
# nodeStatFile=sys.argv[1]


In [3]:
for nodeStatFile in glob.glob("*/*.node_status"):
#nodeStatFile = submitHead+tasksHead+"20250531T113935Z/LSSTCam_runs_nightlyValidation_20250530_d_2025_05_30_DM-50157_20250531T113935Z.node_status"
#nodeStatFile = input("Path of node status file = ")
#    ./20250611T154923Z/jobs/analyzeSingleVisitStar/22e500cc-c984-400a-be69-d182b405bd9d_analyzeSingleVisitStar_2025060900506_.17979981.err
#      Type = "NodeStatus";
#  Node = "22e500cc-c984-400a-be69-d182b405bd9d_analyzeSingleVisitStar_2025060900506_";
#  NodeStatus = 5; /* "STATUS_DONE" */

#    ./20250611T154923Z/jobs/analyzeSingleVisitStarAssociation/040f8daf-bfbd-4574-ba7f-02219bc3fb41_analyzeSingleVisitStarAssociation_7041_.18061387.err
    df = pd.DataFrame(columns=['job name','errors','file'])

    taskID = nodeStatFile.split("/")[0]
    print("task = "+taskID+" | node_status file = "+nodeStatFile)
    dagNodes = glob.glob(taskID+"/*.dag.nodes.log")[0]
    fp_dag = open(dagNodes)
    print("dag.nodes file = "+dagNodes)
    # build nodes ref dictionary
    last_line = ""
    dag_dict = {}
    for ln in fp_dag:
#        print("dag line = "+ln)
        if "DAG Node" in ln:
#            print(ln.split()[2]+" : "+last_line.split("(")[1].split(".")[0])
            dag_dict[ln.split()[2]] = last_line.split("(")[1].split(".")[0]
        else:
            last_line = ln
#    exit
    fp = open(nodeStatFile)
    nextline = False
    jobErrCount = {}
    for jb in jobs:
        jobErrCount[jb]=0
#    print(jobErrCount)
    for ln in fp:
        if ("Type" in ln and "NodeStatus" in ln):
    #        print(ln)
            for ln2 in fp:
                if ("]" in ln2):
                    break
                #print(ln2.strip().split()[0])
                if (ln2.strip().split()[0] == "Node"):
                    element = ln2.strip().split()[2].strip(';').strip('"')
#                    print("element = "+element)
                if (ln2.strip().split()[0] == "NodeStatus"):
                    if ("ERROR" in ln2):
                        element_pieces = element.split("_")
                        element_2_pieces = element.split("-")
                        secondElement = ""
                        try:
                            secondElement = element_pieces[1]
                            lower_path = ""
                            if len(element_pieces)>2 :
                                for ielem in element_pieces[2:] :
                                    lower_path = lower_path + "/" + ielem
#                            print("lower_path = "+lower_path+" len = "+str(len(lower_path)))
#                            secondElement = element_2_pieces[5]
                        except:
                            pass
                        flerr = ""
                        for jb in jobs:
#                            print("jb = "+jb+" | element_pieces[0] = "+element_pieces[0]+" | secondElement = "+secondElement)
#                            print("step1")
                            if jb in element_pieces[0] or nextline:
                                jobErrCount[jb] = jobErrCount[jb]+1
#                                print("jb = "+jb+" | current error count = "+str(jobErrCount[jb]))
#                                print("trying in pieces[0]")
                                try:
#                                    print("path = "+taskID+"/jobs/"+jb+"/"+element_pieces[1]+"/"+jb+"_"+element_pieces[1]+"_"+element_pieces[2]+".*.*")
                                    try:
#                                        print("try1")
                                        try:
#                                            print("1 glob")
                                            for flerr in glob.glob(taskID+"/jobs/"+jb+"/"+element_pieces[1]+"/"+jb+"_"+element_pieces[1]+"_"+element_pieces[2]+".*.*"):
                                                print("flerr = "+flerr)
                                                try:
                                                    the_error = dumpError(flerr)
                                                    if len(the_error)>0 :
                                                        new_row = pd.DataFrame({"job name": [element_pieces[0]], "errors":[dumpError(flerr)], "file":[leadPath+flerr]})
                                                        df = pd.concat([df, new_row], ignore_index=True)
#                                                    print("success on try1")
                                                except:
                                                    pass
#                                            print("2")
                                        except:
                                            pass
                                    
#                                        print("try2: "+taskID+"/jobs/"+jb+"/"+jb+"_"+element_pieces[1]+"_"+element_pieces[2]+".*.*"+" | element = "+element)
                                        try:
#                                            print("2 glob")
                                            if jb in "step1detector" and jobErrCount[jb]>max_step1 :
                                                print("truncating list for jb = "+jb+" | error count = "+str(jobErrCount[jb]))
                                                new_row = pd.DataFrame({"job name": [element_pieces[0]], "errors":["skipped"], "file":[leadPath+taskID+"/jobs/"+jb+"/"+jb+"_"+element_pieces[1]+"_"+element_pieces[2]+".*.*"]})
                                                df = pd.concat([df, new_row], ignore_index=True)
                                                continue
#                                            for flerr in glob.glob(taskID+"/jobs/"+jb+"/"+jb+"_"+element_pieces[1]+"_"+element_pieces[2]+".*.*"):
#                                                print("element = "+element+" | flerr = "+flerr)
#                                                print("dag_dict = ",dag_dict[element])
                                                # examples: 20251110T181218Z/jobs/step1detector/step1detector_105_026.12981746.log
                                                # dag_dict =  12981746
                                                # element = step1detector_105_026 | flerr = 20251110T181218Z/jobs/step1detector/step1detector_105_026.12981746.1.out
#                                                element = step1detector_105_026 | flerr = 20251110T181218Z/jobs/step1detector/step1detector_105_026.12981746.log
#                                                  element = step1detector_105_026 | flerr = 20251110T181218Z/jobs/step1detector/step1detector_105_026.12981746.0.out
                                            flist = []
                                            if jb in "step1detector" :
#                                                print("test98")
                                                flist = [taskID+"/jobs/"+jb+"/"+element+"."+dag_dict[element]+".log"]
                                                flist.append(taskID+"/jobs/"+jb+"/"+element+"."+dag_dict[element]+".0.out")
                                                flist.append(taskID+"/jobs/"+jb+"/"+element+"."+dag_dict[element]+".0.err")
                                            else :
                                                flist = glob.glob(taskID+"/jobs/"+jb+"/"+jb+"_"+element_pieces[1]+"_"+element_pieces[2]+".*.*")
                                            for flerr in flist :
#                                                print("element = "+element+" | flerr = "+flerr)
                                                try:
#                                                    print("before the_error")
                                                    the_error = dumpError(flerr)
#                                                    print("the_error = "+the_error)
                                                    if len(the_error)>0 :
                                                        new_row = pd.DataFrame({"job name": [element_pieces[0]], "errors":[the_error], "file":[leadPath+flerr]})
#                                                        print("created new row")
                                                        df = pd.concat([df, new_row], ignore_index=True)
#                                                    print("success on try2")
                                                except:
                                                    pass
                                        except:
                                            pass
                                    except:
                                        pass
                                except:
                                    pass
#                            print("step2")
                            if jb==secondElement or nextline:
                                jobErrCount[jb] = jobErrCount[jb]+1
                                print("jb = "+jb+" | current error count = "+str(jobErrCount[jb]))
                                try:
                                    if len(lower_path)<1 :
#                                        print("3 glob")
                                        flerr = glob.glob(taskID+"/jobs/"+jb+"/"+element+"*.err")[0]
                                    else:
#                                        print("trying - "+"*/jobs/"+jb+lower_path+element+"*.*")
                                        new_row = None
#                                        print("4 glob")
                                        for flerr in glob.glob(taskID+"/jobs/"+jb+lower_path+element+"*.*"):
#                                            print("flerr = "+flerr)
                                            try:
                                                the_error = dumpError(flerr)
                                                if len(the_error) > 0 :
                                                    new_row = pd.DataFrame({"job name": [secondElement], "errors":[dumpError(flerr)], "file":[leadPath+flerr]})
                                                    df = pd.concat([df, new_row], ignore_index=True)
                                            except:
                                                pass
                                except:
                                    pass
#                            print(element_pieces[1]+"/"+element_pieces[2])
#                            print("*/jobs/step1detector/"+element_pieces[1]+"/step1detector_"+element_pieces[1]+"_"+element_pieces[2]+".*.err")
                        
##                                print(element + ": stat = " + ln2.strip().split()[4].strip('"') + " file = " + flerr )
##                                print(dumpError(flerr))
                        if not nextline:
                            nextline = False # deactivating
                        else:
                            nextline = False
#    print(df)
#    print(tabulate(df, headers='keys', tablefmt='psql'))

    display(HTML(df.to_html()))
    print("\nError count summary:")
    try:
        print(json.dumps(jobErrCount,indent=4))
    except:
        pass
    print("\n\n\n-----------------------------------------------------------------------------------------------------------------------------------\n\n\n")


task = 20251111T172102Z | node_status file = 20251111T172102Z/LSSTCam_runs_DRP_20250421_20250921_d_2025_11_10_DM-53195_20251111T172102Z.node_status
dag.nodes file = 20251111T172102Z/LSSTCam_runs_DRP_20250421_20250921_d_2025_11_10_DM-53195_20251111T172102Z.dag.nodes.log


,job name,errors,file



Error count summary:
{
    "generateEphemerides": 0,
    "fgcmFitCycle": 0,
    "analyzeSingleVisitStar": 0,
    "analyzeSingleVisitStarAssociation": 0,
    "analyzeSingleVisitStarAstrometricRefMatch": 0,
    "analyzeSingleVisitStarPhotometricRefMatch": 0,
    "associateIsolatedStar": 0,
    "coadd": 0,
    "consolidateHealSparsePropertyMaps": 0,
    "consolidateSingleVisitStar": 0,
    "consolidateVisitSummary": 0,
    "finalJob": 0,
    "makeAnalysisSingleVisitStarAssociationMetricTable": 0,
    "makeAnalysisSingleVisitStarAssociationWholeSkyPlot": 0,
    "makeAnalysisSingleVisitStarAstrometricRefMatch": 0,
    "makeAnalysisSingleVisitStarPhotometricRefMatch": 0,
    "makeHealSparsePropertyMaps": 0,
    "makeInitialVisitDetectorTable": 0,
    "makeInitialVisitTable": 0,
    "makeWarpTract": 0,
    "pipetaskInit": 0,
    "plotPropertyMapSurvey": 0,
    "selectDeepCoaddVisits": 0,
    "step1detector": 0
}



-----------------------------------------------------------------------------

,job name,errors,file
0,fgcmFitCycle,error=019a78a8-2709-7f36-9d78-642908cd3d3e_fgcmFitCycle_.$(Cluster).$$([NumJobStarts ?: 0]).out\n,/home/h/huanlin/shared_campaigns_LSST/DM-53195/submit/LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195/20251112T151958Z/jobs/fgcmFitCycle/019a78a8-2709-7f36-9d78-642908cd3d3e_fgcmFitCycle_.sub



Error count summary:
{
    "generateEphemerides": 0,
    "fgcmFitCycle": 1,
    "analyzeSingleVisitStar": 0,
    "analyzeSingleVisitStarAssociation": 0,
    "analyzeSingleVisitStarAstrometricRefMatch": 0,
    "analyzeSingleVisitStarPhotometricRefMatch": 0,
    "associateIsolatedStar": 0,
    "coadd": 0,
    "consolidateHealSparsePropertyMaps": 0,
    "consolidateSingleVisitStar": 0,
    "consolidateVisitSummary": 0,
    "finalJob": 1,
    "makeAnalysisSingleVisitStarAssociationMetricTable": 0,
    "makeAnalysisSingleVisitStarAssociationWholeSkyPlot": 0,
    "makeAnalysisSingleVisitStarAstrometricRefMatch": 0,
    "makeAnalysisSingleVisitStarPhotometricRefMatch": 0,
    "makeHealSparsePropertyMaps": 0,
    "makeInitialVisitDetectorTable": 0,
    "makeInitialVisitTable": 0,
    "makeWarpTract": 0,
    "pipetaskInit": 0,
    "plotPropertyMapSurvey": 0,
    "selectDeepCoaddVisits": 0,
    "step1detector": 0
}



-----------------------------------------------------------------------------

...


Error count summary:
{
    "generateEphemerides": 0,
    "fgcmFitCycle": 1,
    "analyzeSingleVisitStar": 0,
    "analyzeSingleVisitStarAssociation": 0,
    "analyzeSingleVisitStarAstrometricRefMatch": 0,
    "analyzeSingleVisitStarPhotometricRefMatch": 0,
    "associateIsolatedStar": 0,
    "coadd": 0,
    "consolidateHealSparsePropertyMaps": 0,
    "consolidateSingleVisitStar": 0,
    "consolidateVisitSummary": 0,
    "finalJob": 1,
    "makeAnalysisSingleVisitStarAssociationMetricTable": 0,
    "makeAnalysisSingleVisitStarAssociationWholeSkyPlot": 0,
    "makeAnalysisSingleVisitStarAstrometricRefMatch": 0,
    "makeAnalysisSingleVisitStarPhotometricRefMatch": 0,
    "makeHealSparsePropertyMaps": 0,
    "makeInitialVisitDetectorTable": 0,
    "makeInitialVisitTable": 0,
    "makeWarpTract": 0,
    "pipetaskInit": 0,
    "plotPropertyMapSurvey": 0,
    "selectDeepCoaddVisits": 0,
    "step1detector": 0
}



-----------------------------------------------------------------------------

,job name,errors,file



Error count summary:
{
    "generateEphemerides": 0,
    "fgcmFitCycle": 0,
    "analyzeSingleVisitStar": 0,
    "analyzeSingleVisitStarAssociation": 0,
    "analyzeSingleVisitStarAstrometricRefMatch": 0,
    "analyzeSingleVisitStarPhotometricRefMatch": 0,
    "associateIsolatedStar": 0,
    "coadd": 0,
    "consolidateHealSparsePropertyMaps": 0,
    "consolidateSingleVisitStar": 0,
    "consolidateVisitSummary": 0,
    "finalJob": 0,
    "makeAnalysisSingleVisitStarAssociationMetricTable": 0,
    "makeAnalysisSingleVisitStarAssociationWholeSkyPlot": 0,
    "makeAnalysisSingleVisitStarAstrometricRefMatch": 0,
    "makeAnalysisSingleVisitStarPhotometricRefMatch": 0,
    "makeHealSparsePropertyMaps": 0,
    "makeInitialVisitDetectorTable": 0,
    "makeInitialVisitTable": 0,
    "makeWarpTract": 0,
    "pipetaskInit": 0,
    "plotPropertyMapSurvey": 0,
    "selectDeepCoaddVisits": 0,
    "step1detector": 0
}



-----------------------------------------------------------------------------

,job name,errors,file



Error count summary:
{
    "generateEphemerides": 0,
    "fgcmFitCycle": 0,
    "analyzeSingleVisitStar": 0,
    "analyzeSingleVisitStarAssociation": 0,
    "analyzeSingleVisitStarAstrometricRefMatch": 0,
    "analyzeSingleVisitStarPhotometricRefMatch": 0,
    "associateIsolatedStar": 0,
    "coadd": 0,
    "consolidateHealSparsePropertyMaps": 0,
    "consolidateSingleVisitStar": 0,
    "consolidateVisitSummary": 0,
    "finalJob": 1,
    "makeAnalysisSingleVisitStarAssociationMetricTable": 0,
    "makeAnalysisSingleVisitStarAssociationWholeSkyPlot": 0,
    "makeAnalysisSingleVisitStarAstrometricRefMatch": 0,
    "makeAnalysisSingleVisitStarPhotometricRefMatch": 0,
    "makeHealSparsePropertyMaps": 0,
    "makeInitialVisitDetectorTable": 0,
    "makeInitialVisitTable": 0,
    "makeWarpTract": 0,
    "pipetaskInit": 0,
    "plotPropertyMapSurvey": 0,
    "selectDeepCoaddVisits": 0,
    "step1detector": 0
}



-----------------------------------------------------------------------------